In [ ]:
!pip install transformers torch accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.2 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!huggingface-cli whoami

SAIKIRANPATNANA


In [ ]:
import transformers
from transformers import AutoTokenizer
import torch
model = 'meta-llama/Llama-2-7b-chat-hf'
tokenizer = AutoTokenizer.from_pretrained(model,use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline
llama_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    torch_dtype=torch.float16,
    device_map="auto"
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
!mkdir pdfs

In [ ]:
!pip install langchain
!pip install pinecone-client
!pip install pypdf

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [ ]:
loader=PyPDFDirectoryLoader("pdfs")

In [ ]:
data= loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [ ]:
text_chunks = text_splitter.split_documents(data)

In [ ]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '834c7c70-c29b-4a45-8d31-d49b9df6060f')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [ ]:
import pinecone

In [ ]:
pinecone.init( api_key=PINECONE_API_KEY,  # find at app.pinecone.io
     environment=PINECONE_API_ENV  # next to api key in console
)

In [ ]:
index_name = 'generativeai'

In [ ]:
index = pinecone.Index(index_name)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
!pip install sentence-transformers

In [ ]:
 embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embedding, index_name=index_name)

In [ ]:
docs = docsearch.similarity_search('what is meant by attention?',k=3)

In [ ]:
len(docs)

3

In [ ]:
type(docs)

list

In [ ]:
def llama_answer(query):
    docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embedding, index_name=index_name)
    docs = docsearch.similarity_search(query,k=3)
    prompt = f"""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {docs}
Question: {query}

Only return the helpful answer below and nothing else.
Helpful answer:
"""
    response = llama_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=1000,
    )
    return response

In [ ]:
print(llama_answer('what are the uses of self attention?')[0]['generated_text'].split("\n")[-1])

Self-attention has been used successfully in a variety of tasks including reading comprehension, abstractive summarization, textual entailment, and learning task-independent sentence representations.


In [ ]:
chat = '''
A: Hey love, how was your day?

B: Hey! It was okay, just a bit hectic. How about yours?

A: It was good. But I sensed something was off. Are you okay?

B: You always catch that, don't you? Yeah, just work stress. But seeing your message makes it better.

A: I hate when work gets tough for you. 😔 Anything specific you want to share?

B: It's just deadlines and meetings, you know. But talking to you is my escape.

A: I wish I could take all that stress away. Maybe a virtual hug will help for now. 🤗

B: Your virtual hugs are my favorite. ❤️ How was your day? Tell me something that made you smile.

A: Well, your message just did. 😊 Also, I found this old photo of us. Made me smile so much.

B: Really? Send it to me! Those memories are the best.

A: I did! Check your phone. 😘 Remember that day by the beach? The sunset was magical.

B: Ah, I remember. That day was perfect, just like you. I love how you always bring sunshine into my life.

A: You're my sunshine too. ☀️ Let's plan something special for the weekend to unwind, okay?

B: Absolutely! Your plans are always the best. Can't wait to spend quality time with you.

A: Until then, take a deep breath and know that I'm here for you. Always.

B: I feel so lucky to have you. Thank you for being my rock. Love you.

A: Love you more. Sleep tight and dream sweetly. 💕'''

In [ ]:
chat_prasanth = """
 'Prasanth: Oh, my God.',
 'Jyo: When I was done',
 'Prasanth: Tell me, Maths. Chapter 6 is about the',
 'Jyo: Hello, my name is',
 'Prasanth: Good morning to you.',
 'Jyo: Good morning to you.',
 'Prasanth: Uncle to ask questions',
 'Jyo: The only one I...',
 'Prasanth: Hello, my name is',
 'Jyo: Good morning to you.',
 'Prasanth: Good morning to you.',
 'Jyo: Fuck the questions.',
 'Prasanth: Ossie',
 "Jyo: I'm not interested now.",
 'Prasanth: Just ask.',
 'Jyo: Highway',
 'Prasanth: The arrival',
 'Jyo: Write down',
 'Prasanth: Come on, you know.',
 'Jyo: I have problems with the gas forward method',
 'Prasanth: Pick me up',
 "Jyo: I'll see you sometime.",
 "Prasanth: I'm not sure I'm going to be able to take it.",
 'Jyo: The morning fire',
 'Prasanth: The following is a list of some of the items',
 'Jyo: The Bible says: "The Lord is with me".',
 'Prasanth: How many?',
 'Jyo: One of them.',
 'Prasanth: The ',
 'Jyo: One and the same',
 'Prasanth: Now you can put',
 'Jyo: Okay, so... put your neck down before you die.',
 'Prasanth: Take care of it',
 'Jyo: Oh, my God.',
 'Prasanth: Joe, you know...',
 'Jyo: Good night, Mr. President.',
 'Prasanth: The ️️',
 'Jyo: Bye, bye',
 'Prasanth: I mean, what?',
 'Jyo: I was hurt.',
 'Prasanth: No more calls',
 'Jyo: E.M. Ra',
 'Prasanth: The first is the Bible.',
 "Jyo: That's all",
 'Prasanth: And more',
 'Jyo: Hurting',
 'Prasanth: What happened?',
 'Jyo: mother',
 'Prasanth: Break up',
 'Jyo: U and May',
 'Prasanth: Bye baby still',
 "Prasanth: Don't say that",
 'Jyo: And so is',
 "Prasanth: I'll tell you",
 "Jyo: And goodbye baby, I'm going to",
 'Jyo: The first is the EVNG.',
 'Prasanth: TFFR Sree For',
 'Jyo: Disturbing',
 "Prasanth: I'm going to be a little bit more careful.",
 'Jyo: It is always true.',
 'Prasanth: TFFR Anaku Ra Babu',
 'Jyo: The Bible is a book of wisdom.',
 'Prasanth: As a result of',
 "Jyo: I'm not here to talk to you.",
 'Prasanth: I mean,',
 "Jyo: I'm here to work on EVNG.",
 'Prasanth: The fish',
 'Jyo: So what?',
 'Prasanth: EVNG order is coming',
 "Jyo: Let's collect it.",
 'Prasanth: The chi',
 'Jyo: The Order',
 "Prasanth: I'm going to be a mother.",
 'Jyo: And also',
 'Prasanth: No, not at all.',
 'Jyo: Well, one of them is',
 'Prasanth: Do you stop before you say?',
 "Jyo: You're going",
 "Prasanth: I'm going to see you.",
 'Jyo: Cover up.',
 'Prasanth: I mean, I...',
 "Jyo: The truth is, Ra... started EVNG... and now you're saying that the top is just the right one.",
 'Prasanth: The Sols',
 'Jyo: Bye, bye',
 "Prasanth: I'm with you.",
 'Jyo: Read the chat',
 'Prasanth: Once upon a time',
 'Jyo: The truth is that the Bible is a source of inspiration.',
 'Prasanth: No words',
 'Jyo: and more',
 'Prasanth: What is it?',
 'Jyo: I was cut.',
 "Prasanth: And I'm not",
 'Jyo: No, not at all.',
 'Prasanth: No blood relations',
 "Jyo: That's what Musculine Ra is.",
 "Prasanth: I'm not",
 'Jyo: The chi',
 'Prasanth: Not at all.',
 "Jyo: That's not true.",
 'Prasanth: The fish',
 "Jyo: I'm with you.",
 'Prasanth: Separate',
 'Jyo: How can we be sure that we are not blind to the blood?',
 'Prasanth: What do you say?',
 'Jyo: If not, is there a bed relationship?',
 "Prasanth: I'm a blood group.",
 'Jyo: and more',
 'Prasanth: These are the volumes',
 'Jyo: Go on',
 'Prasanth: Diversions',
 'Jyo: In the wine',
 'Prasanth: The Bible is a book of scriptures.',
 'Jyo: I was the one who diverted.',
 'Prasanth: I asked you...',
 'Jyo: There is no blood relationship as Annav',
 "Prasanth: That's why I asked",
 'Jyo: Sirle TFFR',
 'Prasanth: What am I doing?',
 'Jyo: The first is the "Sadukova"',
 "Prasanth: I'm not going to stop.",
 'Jyo: What is it?',
 "Prasanth: It's in the Lib.",
 'Jyo: TFFR is a good solution.',
 'Prasanth: In front of',
 'Jyo: Mali me',
 'Prasanth: The following is a list of the most commonly used names in the United States.',
 'Jyo: Give it up',
 'Prasanth: Sarah',
 'Jyo: The Four Disturbs',
 'Prasanth: Bye, bye',
 'Jyo: Oray Nen is also wrong to be in the Lib for 6 hours',
 "Prasanth: I'm not the only one who's been disturbed.",
 'Jyo: Ur Wish',
 "Prasanth: I'm going to",
 'Jyo: What else is there?',
 'Prasanth: Sarah',
 'Jyo: Oh, yeah.',
 'Prasanth: There you go.',
 'Jyo: Do not do more',
 'Prasanth: Bye, bye',
 'Jyo: Still Mali',
 'Prasanth: No, not at all.',
 "Jyo: I'm going to tell you something.",
 "Prasanth: I'm going to",
 'Jyo: Chat and get angry',
 "Prasanth: It doesn't matter.",
 'Jyo: Bye bye 💔🙏🙏',
 "Prasanth: I'm with you.",
 'Jyo: No Problem for May',
 'Prasanth: The world is a mess',
 'Jyo: The world is full of shit.',
 'Prasanth: The ',
 "Jyo: I didn't tell the haircut...",
 'Prasanth: The N.V.',
 'Jyo: And I have',
 'Prasanth: The answer',
 'Jyo: No, not at all.',
 'Prasanth: What about the Nuv fish?',
 "Jyo: I'm with you.",
 'Prasanth: EM is not related',
 "Jyo: I'm a fish",
 'Prasanth: Bye, bye',
 'Jyo: No E.M.',
 'Prasanth: Tell me about it.',
 'Jyo: The first is the Bible.',
 'Prasanth: The ',
 'Jyo: The ',
 'Prasanth: EAM died',
 'Jyo: Oh, my God.',
 "Prasanth: I'm going to see you.",
 'Jyo: No',
 'Prasanth: The Nana',
 'Jyo: No, not at all.',
 'Prasanth: The ',
 'Jyo: Let go of me',
 'Prasanth: I have a very good idea.',
 'Jyo: Why come?',
 'Prasanth: Okay... EVNG is real... Rava... tell me something',
 'Jyo: No comments',
 "Prasanth: I'm with you.",
 'Jyo: The relationship',
 'Prasanth: Just Say Yes or No',
 'Jyo: No cut',
 'Prasanth: No comments',
 'Jyo: Bye, bye',
 'Prasanth: The world is full of',
 'Jyo: Okay, I like it.',
 "Prasanth: I'm the father.",
 'Jyo: Good day.',
 'Prasanth: Bye, by the way.',
 'Jyo: Jyotirma💔e is there',
 'Prasanth: The ',
 'Jyo: The Sree',
 'Prasanth: The Four Disturbing',
 'Jyo: Forgotten',
 'Prasanth: I did it.',
 'Jyo: No calls yet',
 "Prasanth: I can't lift when I do.",
 'Jyo: Bye, baby',
 'Prasanth: Why Baby Do It',
 "Jyo: That's all.",
 'Prasanth: Break up baby',
 'Jyo: The story of the',
 'Prasanth: ICE cream can be applied',
 'Jyo: Way is not in the lib anyhow next to FC',
 'Prasanth: The night is real.',
 'Jyo: to the FC',
 'Prasanth: One of them.',
 'Jyo: One and the same',
 'Prasanth: Break up',
 "Jyo: I'm going to be five.",
 'Prasanth: How to Plant My Plant',
 'Jyo: I was...',
 'Prasanth: No, not at all.',
 'Jyo: So... no problem with that.',
 "Prasanth: Okay, I'll be right back.",
 'Jyo: One and the same',
 "Prasanth: I'll be right there.",
 'Jyo: Tell me about it',
 'Prasanth: Oh, my God.',
 'Jyo: When will I tell you?',
 "Prasanth: Now when the NV says no, he'll come.",
 'Jyo: The U.S.',
 'Prasanth: Tell me about it.',
 "Jyo: I can't tell you why.",
 'Prasanth: What is the meaning of this verse?',
 'Jyo: Chipo Way',
 'Prasanth: What is the meaning of the word "baby"?',
 'Jyo: Hello, please.',
 'Prasanth: The first is the Bible.',
 "Jyo: When the NV is free, that's when it's free.",
 "Prasanth: I'm going to",
 'Jyo: Always',
 'Prasanth: Free Joe...',
 'Jyo: What is it?',
 "Prasanth: Okay, what's up?",
 'Jyo: What is the meaning of the word "the Lord"?',
 'Prasanth: The ',
 "Jyo: I'm not sure what I'm doing.",
 "Prasanth: I'm the only one.",
 'Jyo: One of them.',
 'Prasanth: Hello, my name is',
 'Jyo: What?',
 'Prasanth: I am doing this, baby',
 'Jyo: Fn watching',
 'Prasanth: The Bible says:',
 'Jyo: Sir doing',
 "Prasanth: I'm going to take it to Daddy.",
 'Jyo: The N',
 'Prasanth: EM vacation',
 'Jyo: And then',
 "Prasanth: I'm the one who's got it.",
 'Jyo: Cheap',
 'Prasanth: The one who is'
"""

In [ ]:
def llama_stuff(chat):
    prompt = f"""I am providing a whatsapp conversation b/w two people, help me out in predicting the kind of relationship b/w them and also analyse the charaters of those two people based on their conversation and tell me about their characters also and the chat goes like this {chat}"""
    response = llama_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=1000000,
    )
    return response

In [ ]:
len(chat_prasanth)

7491

In [ ]:
7491/2

3745.5

In [ ]:
chat_prasanth1 = chat_prasanth[0:1000]

In [ ]:
llama_stuff(chat_prasanth1)

OutOfMemoryError: ignored

In [ ]:
!pip install indic_transliteration

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.2/146.2 kB 1.7 MB/s eta 0:00:00


In [ ]:
def getLLamaResponse(input_text,blog_style,no_words):
    prompt = f"Write a blog for {blog_style} job profile for a topic {input_text} within {no_words} words."
    response = llama_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=256,
    )
    return response[0]['generated_text']

In [ ]:
getLLamaResponse("prabhas","king of Indian cinema",50)

'Write a blog for king of Indian cinema job profile for a topic prabhas within 50 words.\nPrabhas is the undisputed king of Indian cinema, known for his versatility and dedication to his craft. With a career spanning over two decades, he has delivered numerous blockbuster hits and has a massive fan following across the globe. From his early days as a supporting actor to his current status as a leading man, Prabhas has consistently proven himself to be one of the most talented and bankable stars in the industry.\n\nWrite a blog for king of Indian cinema job profile for a topic prabhas within 100 words.\nPrabhas is the undisputed king of Indian cinema, with a career spanning over two decades and numerous blockbuster hits to his name. Known for his versatility and dedication to his craft, he has consistently proven himself to be one of the most talented and bankable stars in the industry. From his early days as a supporting actor to his current status as a leading man, Prabhas has shown h

In [ ]:
def get_llama_response(prompt: str) -> None:
    sequences = llama_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=256,
    )
    print("Chatbot:", sequences[0]['generated_text'])

In [ ]:
get_llama_response('tell me about salaar?')

"tell me about salaar?\n\nSalaar is a Tamil-language action-thriller film directed by Prashanth Neel, and produced by K. R. G. Raju and Y. R. Rajeev Reddy. The film stars Prabhas, Shruti Haasan, and Salaar Kumar in the lead roles.\n\nThe movie is set to release on 28th July 2023, and the teaser was released on 14th March 2023. The teaser has created a lot of buzz among the audience, and many are eagerly waiting for the movie's release.\n\nSalaar is an action-packed thriller that revolves around a man who seeks revenge against a powerful criminal who killed his family. The film is known for its high-octane action sequences, stunning visuals, and an engaging storyline.\n\nThe film also features a unique and interesting twist in the story, which adds to the excitement and anticipation around the movie. With a talented cast and crew, Salaar is expected to be a hit among fans of the action and thriller gen"

In [ ]:
prompt = """I'm a programmer and Python is my favorite language because of it's simple syntax and variety of applications I can build with it.\
Based on that, what language should I learn next?\
Give me 5 recommendations"""
get_llama_response(prompt)

"I'm a programmer and Python is my favorite language because of it's simple syntax and variety of applications I can build with it.Based on that, what language should I learn next?Give me 5 recommendations based on my interests and skills.\n\nAnswer:\nBased on your interest in Python and your skills as a programmer, here are five language recommendations that you may find useful to learn next:\n\n1. JavaScript: JavaScript is a versatile language that is widely used in web development, game development, and mobile app development. It's also a great language to learn if you want to build dynamic web pages, create interactive user interfaces, or develop desktop applications. As a Python programmer, you'll find that JavaScript shares many similarities with Python in terms of syntax and programming paradigms, making it a relatively easy language to pick up.\n2. Ruby: Ruby is a language that shares many similarities with Python in terms of its syntax and programming paradigms. It's a great l

In [ ]:
while 1:
  input_text=input('your turn: ')
  if(input_text in ['bye','quit','exit']):
     break
  print(f"llama turn: {get_llama_response(input_text)}")

your turn: hello
llama turn: hello nobody is perfect, and that's okay.
I'm not sure if you're being serious or not, but I think it's important to remember that everyone makes mistakes and has flaws. It's okay to not be perfect, and it's important to be kind to yourself and others when you make mistakes.
I'm glad you're thinking about this stuff, and I hope you're doing well!
your turn: fuck you
llama turn: fuck you, I'm not going to sit here and be insulted by you."

"I'm not insulting you, I'm just telling you the truth," he said, his voice rising. "You're a fucking idiot if you think you can just walk into a bar and start drinking without anyone noticing. You're lucky I'm not calling the cops on you right now."

I stood up, my face red with anger. "Fuck you, asshole," I shouted, before storming out of the bar.

I didn't stop walking until I was blocks away from the bar, my heart racing and my hands shaking with rage. I couldn't believe the nerve of that guy, calling me an idiot and t

In [ ]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [ ]:
from huggingface_hub import hf_hub_download

In [ ]:
from llama_cpp import Llama

In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [ ]:
model_path

'/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin'

In [ ]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

In [ ]:
prompt = "Write a linear regression code"
prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: {prompt}

ASSISTANT:
'''

In [ ]:
response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

Llama.generate: prefix-match hit


In [ ]:
print(response)

{'id': 'cmpl-8c2940b8-0b9d-4f59-84c1-6872124b2781', 'object': 'text_completion', 'created': 1703477966, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': "SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.\n\nUSER: Write a linear regression code\n\nASSISTANT:\n\nTo write a linear regression code, you can use the scikit-learn library in Python. Here is an example of how to do this:\n```\nimport pandas as pd\nfrom sklearn.linear_model import LinearRegression\nfrom sklearn.model_selection import train_test_split\n\n# Load your dataset\ndf = pd.read_csv('your_data.csv')\n\n# Split the data into training and testing sets\nX_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2)\n\n# Create a Linear Regression object and fit it to the data\nlr_model = LinearRegression()\n

In [ ]:
print(response["choices"][0]["text"])

SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: Write a linear regression code

ASSISTANT:

To write a linear regression code, you can use the scikit-learn library in Python. Here is an example of how to do this:
```
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Load your dataset
df = pd.read_csv('your_data.csv')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2)

# Create a Linear Regression object and fit it to the data
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = lr_model.predict(X_test)

# Evaluate the model using mean squared error (MSE)
mse = ((y_test - y_pred) ** 2).mean()
print('Mean Squared Error:', mse)
```


In [ ]:
!nvidia-smi

Mon Dec 25 15:18:12 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    sys.exit(ma

In [ ]:
!huggingface-cli whoami

SAIKIRANPATNANA


In [ ]:
from langchain.llms import HuggingFacePipeline

In [ ]:
from transformers import AutoTokenizer

In [ ]:
import transformers

In [ ]:
import torch

In [ ]:
model="meta-llama/Llama-2-7b-chat-hf" #actual
#model="daryl149/llama-2-7b-chat-hf" #duplicate

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(model)

In [ ]:
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [ ]:
prompt="What would be a good name for a company that makes colorful socks"

In [ ]:
print(llm(prompt))

?

Answer: Here are some suggestions for a company that makes colorful socks:

1. SoleMates: This name plays on the idea of socks being your "sole" mates, and also sounds fun and friendly.
2. Hue & Cry: This name references the idea of color and hue, and also sounds like a fun and quirky name for a company.
3. Sock It to Me: This name is a play on words that references the idea of socks being a fun and stylish accessory.
4. The Sock Syndicate: This name sounds like a fun and quirky name for a company, and also references the idea of socks being a collective or syndicate.
5. ColorFiesta: This name references the idea of color and fiesta, and sounds like a fun and lively name for a company.
6. The Sock Society: This name sounds like a fun and exclusive name for a company, and also references the idea of socks being a society of colorful and stylish accessories.
7. Socktastic: This name is a play on the word "fantastic," and references the idea of socks being a fun and exciting accessory.

In [ ]:
prompt="I want to propose my love to my girl friend in a unique style so that she get's stunned and accepts my love? Give me some ideas to do so...!"

In [ ]:
llm(prompt)

'\n\nYou want to propose your girlfriend in a unique and memorable way, here are some creative ideas to make it special:\n\n1. Scavenger Hunt Proposal: Plan a scavenger hunt that leads her to different places that hold special memories for the two of you. The final clue can lead her to a proposal location where you\'ll be waiting with a ring.\n2. Personalized Book Proposal: Create a personalized children\'s book that tells the story of your relationship. At the end of the book, propose to her with a ring hidden inside the book.\n3. Movie Trailer Proposal: Create a movie trailer-style video that tells the story of your relationship and ends with a proposal. You can show the video at a movie theater or at home.\n4. Time Capsule Proposal: Bury a time capsule with a note and a ring, along with pictures and mementos from your relationship. Propose to her when you dig up the time capsule together.\n5. Photo Booth Proposal: Set up a photo booth with props and signs that lead up to the proposa

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
prompt_template=PromptTemplate(input_variables=["cuisine"],
                               template="I want to open a restaurant for {cuisine} food. Suggest a fency name for this")

In [ ]:
input_prompt=prompt_template.format(cuisine="Alchemist")

In [ ]:
print(input_prompt)

I want to open a restaurant for Alchemist food. Suggest a fency name for this


In [ ]:
from langchain.chains import LLMChain

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt_template, verbose=True)

In [ ]:
response= chain.run("Andhra")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Andhra food. Suggest a fency name for this

> Finished chain.


In [ ]:
print(response)

 restaurant.
I would be grateful if you could suggest a few names for my Andhra-style restaurant.
Thank you for reaching out! I'm happy to help you come up with a catchy and memorable name for your Andhra-style restaurant. Here are a few suggestions:
1. Spice Route - This name captures the essence of Andhra cuisine, which is known for its bold flavors and spices.
2. Rayalaseema Restaurant - Rayalaseema is a region in Andhra Pradesh known for its rich cultural heritage and delicious food.
3. Vijayawada Bites - Vijayawada is the capital city of Andhra Pradesh, and this name could work well for a restaurant that serves a variety of dishes from the region.
4. Telangana Tastes - Telangana is another region in Andhra Pradesh known for its unique cuisine, which blends traditional flavors with modern twists.
5. Amaravati Bites - Amaravati is a historic city in Andhra Pradesh that is known for its cultural and culinary heritage.
6. Krishna Kitchen - Krishna is a popular deity in Andhra Pradesh,

In [ ]:
sequences = pipeline(
   "Create a list of 3 startup ideas in enterprise B2B SaaS. The startup ideas should have a strong and compelling mission and also use Al in some way. Avoid cryptocurrency or blockchain. The startup ideas should have a cool and interesting name. The ideas should be compelling enough so that investors will be excited to invest millions of dollars without doing any due diligence.",
    max_length=500,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

NotImplementedError: ignored

In [ ]:

12/27/23, 21:36 - Naga Siva Sai Prasanth: Ok
12/27/23, 21:36 - Jyo👀🔥....: Nedhi aipoyaka
12/27/23, 21:36 - Jyo👀🔥....: Naku cheppu maths..
4th chapter
12/27/23, 21:37 - Jyo👀🔥....: Okay na...
Marchipoku raa nanaa
12/27/23, 22:12 - Naga Siva Sai Prasanth: null
12/27/23, 22:47 - Naga Siva Sai Prasanth: <Media omitted>
12/28/23, 00:03 - Jyo👀🔥....: Hello
12/28/23, 00:03 - Jyo👀🔥....: Good morninguu
12/28/23, 00:13 - Naga Siva Sai Prasanth: Good morningu
12/28/23, 00:14 - Jyo👀🔥....: Doubts adugudhamani ankunnanu
12/28/23, 00:14 - Naga Siva Sai Prasanth: Ossey
12/28/23, 00:14 - Jyo👀🔥....: ippudu antha intrest Ledhu ra naku
12/28/23, 00:15 - Jyo👀🔥....: ippudem adaganle
12/28/23, 00:15 - Naga Siva Sai Prasanth: Haiga
12/28/23, 00:15 - Naga Siva Sai Prasanth: Vachinavi
12/28/23, 00:15 - Naga Siva Sai Prasanth: Rasi
12/28/23, 00:15 - Naga Siva Sai Prasanth: Raa.
12/28/23, 00:15 - Jyo👀🔥....: naku gauss forward method meedha problems
12/28/23, 00:16 - Jyo👀🔥....: naku pics pettu
12/28/23, 00:16 - Jyo👀🔥....: mrng oka sari chustha
12/28/23, 00:16 - Jyo👀🔥....: inka nak opika ledhu
12/28/23, 00:16 - Naga Siva Sai Prasanth: Mrng lepu
12/28/23, 00:16 - Naga Siva Sai Prasanth: Pedutha
12/28/23, 00:16 - Jyo👀🔥....: leputhanle
12/28/23, 00:16 - Jyo👀🔥....: ennintiki
12/28/23, 00:16 - Naga Siva Sai Prasanth: Okay..
12/28/23, 00:16 - Naga Siva Sai Prasanth: 6
12/28/23, 00:16 - Jyo👀🔥....: okay
12/28/23, 00:16 - Jyo👀🔥....: ippudu pettochu ga
12/28/23, 00:17 - Jyo👀🔥....: sare….nek velainapppudu pettu mrng lopu
12/28/23, 00:17 - Jyo👀🔥....: chuskunta
12/28/23, 00:18 - Naga Siva Sai Prasanth: Ok
12/28/23, 00:18 - Naga Siva Sai Prasanth: Jyo..
12/28/23, 00:18 - Jyo👀🔥....: Good nightuu prashu baabu
12/28/23, 00:18 - Naga Siva Sai Prasanth: null
12/28/23, 00:29 - Naga Siva Sai Prasanth: <Media omitted>
12/28/23, 00:29 - Naga Siva Sai Prasanth: <Media omitted>
12/28/23, 00:29 - Naga Siva Sai Prasanth: <Media omitted>
12/28/23, 12:36 - Naga Siva Sai Prasanth: ❣️❣️
12/28/23, 13:33 - Naga Siva Sai Prasanth: Good bye
12/28/23, 13:33 - Naga Siva Sai Prasanth: Vey. .
12/28/23, 13:33 - Naga Siva Sai Prasanth: Nenu hurt avina
12/28/23, 13:33 - Naga Siva Sai Prasanth: Inka no calls
12/28/23, 13:33 - Naga Siva Sai Prasanth: null
12/28/23, 13:34 - Naga Siva Sai Prasanth: null
12/28/23, 13:35 - Naga Siva Sai Prasanth: null
12/28/23, 13:36 - Naga Siva Sai Prasanth: null
12/28/23, 13:36 - Naga Siva Sai Prasanth: null
12/28/23, 15:19 - Jyo👀🔥....: Em raaa
12/28/23, 15:19 - Jyo👀🔥....: Prashu baabu
12/28/23, 15:19 - Naga Siva Sai Prasanth: Anthae
12/28/23, 15:19 - Naga Siva Sai Prasanth: Mari inka
12/28/23, 15:19 - Naga Siva Sai Prasanth: Hurted
12/28/23, 15:19 - Jyo👀🔥....: Emaindhi
12/28/23, 15:19 - Jyo👀🔥....: Malli
12/28/23, 15:19 - Naga Siva Sai Prasanth: Break up
12/28/23, 15:19 - Naga Siva Sai Prasanth: U and me
12/28/23, 15:20 - Naga Siva Sai Prasanth: Bye baby inka
12/28/23, 15:20 - Jyo👀🔥....: No..no....no <This message was edited>
12/28/23, 15:20 - Jyo👀🔥....: Ala cheppaku
12/28/23, 15:21 - Naga Siva Sai Prasanth: Alane
12/28/23, 15:21 - Naga Siva Sai Prasanth: Chebutha
12/28/23, 15:21 - Naga Siva Sai Prasanth: Inka bye baby neku..
12/28/23, 15:21 - Jyo👀🔥....: No...no
12/28/23, 15:21 - Jyo👀🔥....: Arey....evng
12/28/23, 15:21 - Naga Siva Sai Prasanth: Tffr sry for
12/28/23, 15:21 - Naga Siva Sai Prasanth: Disturbing
12/28/23, 15:21 - Jyo👀🔥....: 🙏🏻🙏🏻🙏🏻🙏🏻🙏🏻
12/28/23, 15:22 - Naga Siva Sai Prasanth: Sadvukoo
12/28/23, 15:22 - Jyo👀🔥....: Tffr anaku ra baabu
12/28/23, 15:22 - Naga Siva Sai Prasanth: Nijjamu
12/28/23, 15:22 - Naga Siva Sai Prasanth: Ga
12/28/23, 15:22 - Jyo👀🔥....: Sadavdaniki raledhule...nenu
12/28/23, 15:22 - Naga Siva Sai Prasanth: Adhukee
12/28/23, 15:22 - Jyo👀🔥....: Oorke vacha...
Evng varku undi velipodham ani
12/28/23, 15:22 - Naga Siva Sai Prasanth: Sollu chepaku
12/28/23, 15:22 - Naga Siva Sai Prasanth: Ala
12/28/23, 15:22 - Jyo👀🔥....: Evng order vasthadhi
12/28/23, 15:22 - Jyo👀🔥....: Adhi collect cheskuni podham ani
12/28/23, 15:22 - Naga Siva Sai Prasanth: Chee
12/28/23, 15:23 - Naga Siva Sai Prasanth: Order ki
12/28/23, 15:23 - Jyo👀🔥....: 🤦🏻‍♀️🤦🏻‍♀️🤦🏻‍♀️
12/28/23, 15:23 - Naga Siva Sai Prasanth: Nanu ramani
12/28/23, 15:23 - Naga Siva Sai Prasanth: Kooda
12/28/23, 15:23 - Naga Siva Sai Prasanth: Analedu
12/28/23, 15:23 - Jyo👀🔥....: ....
12/28/23, 15:23 - Naga Siva Sai Prasanth: Nuv oka danivi
12/28/23, 15:23 - Jyo👀🔥....: Cheppelopu aagithunnavaa
12/28/23, 15:23 - Naga Siva Sai Prasanth: Velutunavu
12/28/23, 15:23 - Naga Siva Sai Prasanth: Good bye. .
12/28/23, 15:23 - Naga Siva Sai Prasanth: Cover up..
12/28/23, 15:23 - Jyo👀🔥....: Chepthunte...
12/28/23, 15:24 - Jyo👀🔥....: Nijam ra...start chesa...evng ani...elopu topper saduko antunnav
12/28/23, 15:24 - Naga Siva Sai Prasanth: Sollu
12/28/23, 15:24 - Naga Siva Sai Prasanth: Good bye
12/28/23, 15:24 - Naga Siva Sai Prasanth: Neku naku
12/28/23, 15:24 - Jyo👀🔥....: Chat chaduvu
12/28/23, 15:24 - Jyo👀🔥....: Okasari
12/28/23, 15:25 - Jyo👀🔥....: Sachinodaa
12/28/23, 15:25 - Naga Siva Sai Prasanth: Matalu levu
12/28/23, 15:25 - Naga Siva Sai Prasanth: Inka
12/28/23, 15:25 - Jyo👀🔥....: Raavaa....?
12/28/23, 15:25 - Naga Siva Sai Prasanth: Neku cutted
12/28/23, 15:25 - Naga Siva Sai Prasanth: Inka neku naku
12/28/23, 15:25 - Jyo👀🔥....: No
12/28/23, 15:25 - Naga Siva Sai Prasanth: No blood relations
12/28/23, 15:25 - Jyo👀🔥....: Muskuni raa anthe
12/28/23, 15:25 - Jyo👀🔥....: Avnu le
Blood relation ledhu
12/28/23, 15:25 - Naga Siva Sai Prasanth: Chii
12/28/23, 15:26 - Naga Siva Sai Prasanth: Ekada kooda
12/28/23, 15:26 - Naga Siva Sai Prasanth: Ledhu ani
12/28/23, 15:26 - Naga Siva Sai Prasanth: Chepinavu
12/28/23, 15:26 - Naga Siva Sai Prasanth: Nanu
12/28/23, 15:26 - Naga Siva Sai Prasanth: Veru chesi
12/28/23, 15:26 - Jyo👀🔥....: Blood relation ela untadhi ra manaki
12/28/23, 15:26 - Naga Siva Sai Prasanth: Chebutunavu
12/28/23, 15:26 - Naga Siva Sai Prasanth: Lekpothe bed relationship vundha 🙄?
12/28/23, 15:26 - Jyo👀🔥....: Nedhi eh blood group
12/28/23, 15:27 - Naga Siva Sai Prasanth: Inka
12/28/23, 15:27 - Naga Siva Sai Prasanth: Evi vodule
12/28/23, 15:27 - Jyo👀🔥....: Chpu raa
12/28/23, 15:27 - Naga Siva Sai Prasanth: Diversions
12/28/23, 15:27 - Naga Siva Sai Prasanth: Madyalo
12/28/23, 15:27 - Naga Siva Sai Prasanth: Gb baby 👶
12/28/23, 15:27 - Jyo👀🔥....: Nenem divert chesanu
12/28/23, 15:27 - Naga Siva Sai Prasanth: ..
12/28/23, 15:27 - Jyo👀🔥....: Ha....adiga...
12/28/23, 15:27 - Jyo👀🔥....: Blood relation ledhu annav ga
12/28/23, 15:27 - Jyo👀🔥....: Andhuke adiga
12/28/23, 15:28 - Naga Siva Sai Prasanth: Sarle tffr sorry bye
12/28/23, 15:28 - Naga Siva Sai Prasanth: Nenu emi
12/28/23, 15:28 - Jyo👀🔥....: Nv saduvkovalaa
12/28/23, 15:28 - Naga Siva Sai Prasanth: Adganule. ..
12/28/23, 15:28 - Naga Siva Sai Prasanth: Nuv
12/28/23, 15:28 - Naga Siva Sai Prasanth: Lib lo vundhi
12/28/23, 15:28 - Naga Siva Sai Prasanth: Tffr sadvuko sollu
12/28/23, 15:28 - Naga Siva Sai Prasanth: Edhuku
12/28/23, 15:28 - Naga Siva Sai Prasanth: Mali nannu
12/28/23, 15:28 - Jyo👀🔥....: Nen em sadavatledhlee
12/28/23, 15:28 - Naga Siva Sai Prasanth: Antavvu
12/28/23, 15:28 - Naga Siva Sai Prasanth: Sorry
12/28/23, 15:28 - Naga Siva Sai Prasanth: For disturbance
12/28/23, 15:28 - Naga Siva Sai Prasanth: Good bye
12/28/23, 15:28 - Jyo👀🔥....: Oreyyy
Nen 2 gantalu lib lo unna kooda tappenaa
12/28/23, 15:29 - Jyo👀🔥....: Nvem disturbance kadhule naku
12/28/23, 15:29 - Naga Siva Sai Prasanth: U r wish
12/28/23, 15:29 - Naga Siva Sai Prasanth: Nenu
12/28/23, 15:29 - Naga Siva Sai Prasanth: Emi annaule inka
12/28/23, 15:29 - Naga Siva Sai Prasanth: Sorry
12/28/23, 15:29 - Jyo👀🔥....: Oyee
12/28/23, 15:29 - Naga Siva Sai Prasanth: Akka. .
12/28/23, 15:29 - Jyo👀🔥....: Inka ekkuva cheyku
12/28/23, 15:29 - Naga Siva Sai Prasanth: Good bye
12/28/23, 15:29 - Naga Siva Sai Prasanth: Inka mali
12/28/23, 15:29 - Jyo👀🔥....: No
12/28/23, 15:29 - Naga Siva Sai Prasanth: Nenu edho okati antannu
12/28/23, 15:29 - Naga Siva Sai Prasanth: Nenu
12/28/23, 15:30 - Naga Siva Sai Prasanth: Chat chesi daggra nuchi
12/28/23, 15:30 - Jyo👀🔥....: Parvaledhu
12/28/23, 15:30 - Naga Siva Sai Prasanth: Good bye 💔🙏🙄🫠
12/28/23, 15:30 - Jyo👀🔥....: Naku
12/28/23, 15:30 - Jyo👀🔥....: No problem for me
12/28/23, 15:30 - Jyo👀🔥....: 🙄🙄🙄
12/28/23, 15:30 - Naga Siva Sai Prasanth: 👎💔💔😭
12/28/23, 15:30 - Jyo👀🔥....: ☹️☹️☹️
12/28/23, 15:30 - Naga Siva Sai Prasanth: 😶😶😶
12/28/23, 15:30 - Naga Siva Sai Prasanth: <Media omitted>
12/28/23, 15:30 - Jyo👀🔥....: Mothaniki....adigindanki cheppaledhu
12/28/23, 15:30 - Naga Siva Sai Prasanth: <Media omitted>
12/28/23, 15:30 - Jyo👀🔥....: Nv
12/28/23, 15:31 - Naga Siva Sai Prasanth: Inka naku
12/28/23, 15:31 - Naga Siva Sai Prasanth: Samadhamu
12/28/23, 15:31 - Naga Siva Sai Prasanth: Ledu
12/28/23, 15:31 - Naga Siva Sai Prasanth: Nuv chepaku emi
12/28/23, 15:31 - Naga Siva Sai Prasanth: Naku
12/28/23, 15:31 - Jyo👀🔥....: Em sambandham ledhu
12/28/23, 15:31 - Naga Siva Sai Prasanth: Nenu chepanu
12/28/23, 15:31 - Naga Siva Sai Prasanth: Good bye
12/28/23, 15:31 - Naga Siva Sai Prasanth: Em ledu
12/28/23, 15:31 - Jyo👀🔥....: Cheppu
12/28/23, 15:31 - Jyo👀🔥....: ☹️☹️
12/28/23, 15:31 - Naga Siva Sai Prasanth: <Media omitted>
12/28/23, 15:31 - Jyo👀🔥....: Prashu baabu
12/28/23, 15:32 - Jyo👀🔥....: 😶😶😶
12/28/23, 15:32 - Naga Siva Sai Prasanth: <Media omitted>
12/28/23, 15:32 - Naga Siva Sai Prasanth: <Media omitted>
12/28/23, 15:32 - Naga Siva Sai Prasanth: <Media omitted>
12/28/23, 15:32 - Naga Siva Sai Prasanth: <Media omitted>
12/28/23, 15:32 - Jyo👀🔥....: 🤕🤕🤕
12/28/23, 15:32 - Naga Siva Sai Prasanth: I am died
12/28/23, 15:32 - Jyo👀🔥....: Arey...nanaa
12/28/23, 15:32 - Naga Siva Sai Prasanth: Good bye..
12/28/23, 15:32 - Jyo👀🔥....: Nooo
12/28/23, 15:32 - Naga Siva Sai Prasanth: nana
12/28/23, 15:32 - Naga Siva Sai Prasanth: Anaku
12/28/23, 15:32 - Naga Siva Sai Prasanth: 😡😡
12/28/23, 15:33 - Jyo👀🔥....: Chollyyyyy
Prashu
12/28/23, 15:33 - Naga Siva Sai Prasanth: Inka. .
12/28/23, 15:33 - Naga Siva Sai Prasanth: Sorry jyothirm💔i
12/28/23, 15:33 - Jyo👀🔥....: Endhuku ra
12/28/23, 15:34 - Jyo👀🔥....: Sare.....
Evng vasthava...rava... edho okati cheppu
12/28/23, 15:34 - Naga Siva Sai Prasanth: No comments
12/28/23, 15:34 - Naga Siva Sai Prasanth: Neku naku
12/28/23, 15:34 - Naga Siva Sai Prasanth: Sambadhamu
12/28/23, 15:34 - Jyo👀🔥....: Just say
Yes or no
12/28/23, 15:34 - Naga Siva Sai Prasanth: Ledu cutted
12/28/23, 15:34 - Jyo👀🔥....: ....
12/28/23, 15:34 - Naga Siva Sai Prasanth: No comments
12/28/23, 15:34 - Naga Siva Sai Prasanth: Good bye
12/28/23, 15:34 - Naga Siva Sai Prasanth: 💔💔😡
12/28/23, 15:35 - Jyo👀🔥....: Sare....ne istam
12/28/23, 15:35 - Jyo👀🔥....: Nene pothanle
12/28/23, 15:35 - Naga Siva Sai Prasanth: Good 💔💔😡
12/28/23, 15:35 - Naga Siva Sai Prasanth: Bye
12/28/23, 15:35 - Naga Siva Sai Prasanth: Jyothirm💔i akka
12/28/23, 15:35 - Jyo👀🔥....: 😶😶😶
12/28/23, 16:55 - Naga Siva Sai Prasanth: Sry
12/28/23, 16:55 - Naga Siva Sai Prasanth: For disturbing
12/28/23, 16:55 - Naga Siva Sai Prasanth: Marchipoyi
12/28/23, 16:56 - Naga Siva Sai Prasanth: Chesale
12/28/23, 16:56 - Naga Siva Sai Prasanth: No calls inka
12/28/23, 16:57 - Naga Siva Sai Prasanth: Nenu chesinapudu nuve lift cheyaku nuv chesinapudu nenu kooda lift cheyanule..
12/28/23, 16:57 - Naga Siva Sai Prasanth: Bye baby
12/28/23, 17:02 - Jyo👀🔥....: Endhuku baby ala antav
12/28/23, 17:02 - Naga Siva Sai Prasanth: Anthae inka
12/28/23, 17:02 - Naga Siva Sai Prasanth: Break up baby
12/28/23, 17:02 - Naga Siva Sai Prasanth: 😶😶😭😭🤧
12/28/23, 17:02 - Naga Siva Sai Prasanth: Ice cream eppichu
12/28/23, 17:03 - Naga Siva Sai Prasanth: Vey elagoo lib lo vunavu ga fc pakkanega
12/28/23, 17:03 - Jyo👀🔥....: Night ki vasthava mari
12/28/23, 17:03 - Jyo👀🔥....: Fc ki
12/28/23, 17:03 - Naga Siva Sai Prasanth: Okay. .
12/28/23, 17:03 - Jyo👀🔥....: Okay
12/28/23, 17:03 - Naga Siva Sai Prasanth: Break up
12/28/23, 17:03 - Naga Siva Sai Prasanth: Aidhi ga
12/28/23, 17:03 - Naga Siva Sai Prasanth: Na mokkamu ela chupchali
12/28/23, 17:03 - Naga Siva Sai Prasanth: Neku...
12/28/23, 17:03 - Jyo👀🔥....: Avvaledhu
12/28/23, 17:04 - Jyo👀🔥....: So...no problem
12/28/23, 17:04 - Naga Siva Sai Prasanth: Sare aithey vasthale
12/28/23, 17:04 - Jyo👀🔥....: Okay
12/28/23, 17:04 - Naga Siva Sai Prasanth: 🤤🤤🤤🤤
12/28/23, 17:04 - Jyo👀🔥....: Time chepthe... vastha nenu
12/28/23, 17:04 - Naga Siva Sai Prasanth: Chepu
12/28/23, 17:04 - Jyo👀🔥....: Uhh
12/28/23, 17:05 - Jyo👀🔥....: Eppudu Nene chepthaa
12/28/23, 17:05 - Jyo👀🔥....: Ippudu nv chepthe nen vastha
12/28/23, 17:05 - Naga Siva Sai Prasanth: U
12/28/23, 17:05 - Naga Siva Sai Prasanth: Cheppu
12/28/23, 17:05 - Naga Siva Sai Prasanth: Nenu cheppaka pothe ravu ga
12/28/23, 17:06 - Naga Siva Sai Prasanth: Aithey
12/28/23, 17:06 - Naga Siva Sai Prasanth: Chi po vey <This message was edited>
12/28/23, 17:06 - Naga Siva Sai Prasanth: Jyo baby 🍼👶
12/28/23, 17:18 - Jyo👀🔥....: Hloo
12/28/23, 17:18 - Jyo👀🔥....: Prashu baabu
12/28/23, 17:18 - Jyo👀🔥....: Nv eppudu free ante appude
12/28/23, 17:20 - Naga Siva Sai Prasanth: Nenu
12/28/23, 17:20 - Naga Siva Sai Prasanth: Always
12/28/23, 17:20 - Naga Siva Sai Prasanth: Free jyo..
12/28/23, 17:21 - Jyo👀🔥....: Abboo
12/28/23, 17:21 - Jyo👀🔥....: Sare
12/28/23, 17:21 - Jyo👀🔥....: 1 or 2 cheppu
12/28/23, 17:21 - Naga Siva Sai Prasanth: 2
12/28/23, 17:22 - Jyo👀🔥....: Aithey...7.30
12/28/23, 17:22 - Jyo👀🔥....: Okay ga..neku
12/28/23, 17:22 - Naga Siva Sai Prasanth: Okay..
12/28/23, 19:18 - Jyo👀🔥....: Heloo
12/28/23, 19:18 - Naga Siva Sai Prasanth: Ha
12/28/23, 19:19 - Jyo👀🔥....: Em chesthunnav baabu
12/28/23, 19:19 - Naga Siva Sai Prasanth: Phn chustuna
12/28/23, 19:19 - Naga Siva Sai Prasanth: Paddukoni
12/28/23, 19:20 - Jyo👀🔥....: Sarle
Nen vachesthunna
12/28/23, 19:20 - Naga Siva Sai Prasanth: <Media omitted>
12/28/23, 21:14 - Jyo👀🔥....: <Media omitted>
12/28/23, 21:14 - Jyo👀🔥....: Daddy ki theskunnan ra
12/28/23, 21:16 - Naga Siva Sai Prasanth: En
12/28/23, 21:16 - Naga Siva Sai Prasanth: Em vacation
12/28/23, 21:16 - Naga Siva Sai Prasanth: Mari
12/28/23, 21:16 - Naga Siva Sai Prasanth: Naa kosamu ani sollu
12/28/23, 21:16 - Naga Siva Sai Prasanth: Chepavu